# Load multiband raster data via stream API

In [1]:
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.colors as mcolors
import geoengine as ge
import warnings

# setting `clip_to_query_rectangle=True` may output a warning, but this is a false positive from the underlying library
warnings.simplefilter(action='ignore', category=FutureWarning)

## Initialize Geo Engine

In [2]:
ge.initialize("http://localhost:3030/api")

## Define workflow of a stacked raster of ndvi and land cover

In [3]:
workflow_ndvi = ge.workflow_builder.operators.GdalSource("ndvi")
workflow_land_cover = ge.workflow_builder.operators.GdalSource("land_cover")
workflow_stack = ge.workflow_builder.operators.RasterStacker(sources=[workflow_ndvi, workflow_land_cover])
workflow_downsample = ge.workflow_builder.operators.Downsampling(source_operator=workflow_stack, output_method='resolution', output_x=0.5, output_y=0.5)


workflow = ge.register_workflow(workflow_downsample)
workflow

c6928e36-90bd-5152-ac84-ee3c075d83ba

## Define a query rectangle

In [4]:
time_start = datetime.strptime('2014-04-01T12:00:00.000Z', "%Y-%m-%dT%H:%M:%S.%f%z")
time_end = datetime.strptime('2014-06-01T12:00:00.000Z', "%Y-%m-%dT%H:%M:%S.%f%z")
bbox = ge.QueryRectangle(
    ge.BoundingBox2D(-180.0, -90.0, 180.0, 90.0),
    ge.TimeInterval(time_start, time_end),
    # ge.SpatialResolution(0.5, 0.5), # this was the original reslution but in a stream you need to use an operator for that
)

# Query single tiles, merge them locally and output them as a single `DataArray`

In [5]:
array = await workflow.raster_stream_into_xarray(bbox, bands=[0, 1], clip_to_query_rectangle=True)

array.isel(band=0).plot(col="time")
plt.suptitle("Band 0 (NDVI) over time", y=1.05)
plt.show()

array.isel(band=1).plot(col="time")
plt.suptitle("Band 1 (Land cover) over time", y=1.05)
plt.show()


ConnectionClosedError: sent 1002 (protocol error); no close frame received

## Output structure of the array

In [ ]:
array.to_dataset(name = "Multi-Band")

## Define a multi-band raster as a an RGB composite

In [ ]:
workflow = ge.register_workflow({
    "type": "Raster",
    "operator": {
        "type": "RasterStacker",
        "params": {
            "renameBands": {
                "type": "default"
            }                    
        },
        "sources": {
            "rasters": [
                {
                    "type": "GdalSource",
                    "params": {
                        "data": "ne2_raster_red"
                    }
                }, {
                    "type": "GdalSource",
                    "params": {
                        "data": "ne2_raster_green"
                    }
                }, {
                    "type": "GdalSource",
                    "params": {
                        "data": "ne2_raster_blue"
                    }
                }
            ]
        }
    }
})

array = await workflow.raster_stream_into_xarray(bbox, bands=[0, 1, 2], clip_to_query_rectangle=True)

array.to_dataset(name = "rgb")

## Plot the bands individually

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(6, 9))

cmap = plt.get_cmap('Reds')
norm = mcolors.Normalize(vmin=0, vmax=255)
array.isel(band=0).plot(cmap=cmap, ax=axs[0])
axs[0].set_title("Natural Earth red")

cmap = plt.get_cmap('Greens')
norm = mcolors.Normalize(vmin=0, vmax=255)
array.isel(band=1).plot(cmap=cmap, ax=axs[1])
axs[1].set_title("Natural Earth green")

cmap = plt.get_cmap('Blues')
norm = mcolors.Normalize(vmin=0, vmax=255)
array.isel(band=2).plot(cmap=cmap, ax=axs[2])
axs[2].set_title("Natural Earth blue")

plt.tight_layout()
plt.show()

## Plot the RGB composite

In [ ]:
plt.figure(figsize=(6,3))

array.fillna(0).isel(time=0, drop=True).astype("int").plot.imshow(rgb="band"),

plt.title("Natural Earth RGB composite")
plt.show()
